# 编程作业
- 使用第三课的编程作业中的格子迷宫环境。需要实现以下算法。 (在更新算法时，必须时无模型的，即不能使用 , ), 可以与第三课的作业进行对比
- (在策略) 使用增量式蒙特卡洛的方法，计算随机策略的值函数。
- 使用常量步长，计算随机策略的值函数。对比不同的值函数初始化下 (比如把值函数的初始值设为一个均值很大的正态分布等常量步长和一般增量式蒙特卡洛的收敛性。
- 实现 GLIE 蒙特卡洛优化算法，求出最优策略。
- 尝试在蒙特卡洛方法中使用贪婪的策略提升,是否能找出最优策略?跟什么有关?
- (离策略) 选择行为策略 μ 为随机策略，然后使用增量式离策略每次拜访蒙特卡洛优化算法，求出此时的最优策略。

In [1]:
import numpy as np
from code3_DP import Env
import time
import copy
WORLD_SIZE = 5
N_ACTIONS = 4  # [North, South, West, East]
gamma = 0.9

In [2]:
class Agent:
    def random_policy(self, s):
        return np.random.randint(0, 4)

In [3]:
def reset(self):
    return np.random.randint(0, WORLD_SIZE, size=2)

Agent.reset = reset

In [4]:
def MCSample(env, policy, init_s, T=100):
    episode = []
    s = init_s
    for t in range(T):
        a = policy(s)
        s_n, r = env.step(s, a)
        episode.append(dict(s=s, a=a, r=r))
        s = s_n
    return episode

In [5]:
env = Env()
agent = Agent()
MCSample(env, agent.random_policy, [0, 0], T=10)

[{'s': [0, 0], 'a': 2, 'r': -1.0},
 {'s': [0, 0], 'a': 0, 'r': -1.0},
 {'s': [0, 0], 'a': 2, 'r': -1.0},
 {'s': [0, 0], 'a': 2, 'r': -1.0},
 {'s': [0, 0], 'a': 0, 'r': -1.0},
 {'s': [0, 0], 'a': 0, 'r': -1.0},
 {'s': [0, 0], 'a': 0, 'r': -1.0},
 {'s': [0, 0], 'a': 2, 'r': -1.0},
 {'s': [0, 0], 'a': 3, 'r': 0.0},
 {'s': [0, 1], 'a': 2, 'r': 10.0}]

In [6]:
def random_policy_evaluation(max_time_step=1000, T=200, V_init=None, alpha=None):
    """
    这里alpha常量步长算法中的步长。如果alpha=None,表示不使用常量步长。
    """
    if V_init is None:
        V = np.zeros((WORLD_SIZE, WORLD_SIZE), dtype=np.float32)
    else:
        V = V_init
    env = Env()
    agent = Agent()
    if alpha is None:
        s_count = np.zeros((WORLD_SIZE, WORLD_SIZE), dtype=np.float32)
    for _ in range(max_time_step):

        init_s = agent.reset()
        episode = MCSample(env, agent.random_policy, init_s, T=T)
        g = 0
        drop_step = 0
        # 为了降低计算复杂度，可以反向遍历，可以只用维护一个g变量
        for transition in episode[::-1]:
            s, a, r = transition['s'], transition['a'], transition['r']
            g = r + gamma * g
            drop_step += 1
            if drop_step > T / 2:
                if alpha is None:
                    s_count[s[0], s[1]] += 1
                    V[s[0], s[1]] = V[s[0], s[1]] + (g - V[s[0], s[1]]) / s_count[s[0], s[1]]
                else:
                    V[s[0], s[1]] = V[s[0], s[1]] + (g - V[s[0], s[1]]) * alpha
    return V

In [7]:
random_V = random_policy_evaluation(2000, 200)
print(random_V)

[[ 3.3911808   8.782748    4.6850076   5.3397326   1.4900396 ]
 [ 1.5520037   2.921136    2.3135724   2.0685825   0.65293974]
 [ 0.05544021  0.76511365  0.71274924  0.43168846 -0.31611654]
 [-0.97082937 -0.4022827  -0.32979733 -0.5776283  -1.1455901 ]
 [-1.8563095  -1.3187003  -1.1959155  -1.391154   -1.8886324 ]]


![](http://ww4.sinaimg.cn/large/006tNc79ly1g5ghw0qhnmj30wk0erwio.jpg)



In [8]:
def random_policy_evaluation(max_time_step=1000, T=200, V_init=None, alpha=None):
    """
    这里alpha常量步长算法中的步长。如果alpha=None,表示不使用常量步长。
    """
    if V_init is None:
        V = np.zeros((WORLD_SIZE, WORLD_SIZE), dtype=np.float32)
    else:
        V = V_init
    env = Env()
    agent = Agent()
    if alpha is None:
        s_count = np.zeros((WORLD_SIZE, WORLD_SIZE), dtype=np.float32)
    for _ in range(max_time_step):

        init_s = agent.reset()
        episode = MCSample(env, agent.random_policy, init_s, T=T)
        g = 0
        drop_step = 0
        # 为了降低计算复杂度，可以反向遍历，可以只用维护一个g变量
        for transition in episode[::-1]:
            s, a, r = transition['s'], transition['a'], transition['r']
            g = r + gamma * g
            drop_step += 1
            if drop_step > T / 2:
                if alpha is None:
                    s_count[s[0], s[1]] += 1
                    V[s[0], s[1]] = V[s[0], s[1]] + (g - V[s[0], s[1]]) / s_count[s[0], s[1]]
                else:
                    V[s[0], s[1]] = V[s[0], s[1]] + (g - V[s[0], s[1]]) * alpha
    return V

In [9]:
random_V = random_policy_evaluation(2000, 200)
print(random_V)


[[ 3.4674397   8.80356     4.487427    5.483228    1.613079  ]
 [ 1.5144688   2.9913697   2.313193    1.9929569   0.6959599 ]
 [ 0.01889825  0.7167087   0.6872955   0.4519823  -0.33090007]
 [-0.96016335 -0.44564405 -0.3579836  -0.5370258  -1.1255953 ]
 [-1.8871627  -1.3777544  -1.2457054  -1.3900825  -1.8995789 ]]


![](http://ww1.sinaimg.cn/large/006tNc79ly1g5ghxws3jlj30m6055gmk.jpg)

In [10]:
random_V = random_policy_evaluation(2000, 200, alpha=0.1)
print(random_V)

[[ 4.4889317   9.127194    4.849655    5.513523    0.9619732 ]
 [ 1.1219475   4.2433023   4.5203247   2.9436836   0.15085517]
 [-0.3457649   2.0828152   1.98433     0.62299305 -0.05702705]
 [ 0.32030225 -0.44546118 -0.30470967 -1.1299189  -1.1479304 ]
 [-0.02236242 -0.5579249  -0.29912665 -0.78376895 -2.4887552 ]]


可以看出常量步长下的误差更大


接下来我们开始实现GLIE的MC优化算法，来寻找最优的策略。和之前的算法不同，GLIE有这么几个关键点：

- 1 为了能够从值函数导出策略，我们需要更新的是Q函数，而不是V函数。
- 2 我们需要保证探索（比如 ε -greedy)
- 3 随着训练的进行，策略需要收敛到贪婪策略（比如，不断地衰减 ε 的值)
实现GLIE如下

首先我们先实现一个通过当前Q值和 ε 的值返回一个策略的函数

In [11]:
def epsilon_greedy_policy(self, Q, epsilon):
    """Q: [x, y, a]
    """ 
    def _policy(s):
        Qs = Q[s[0], s[1]]
        if np.random.uniform(0, 1) > epsilon:
            return np.argmax(Qs)
        else:
            return np.random.randint(0, 4)
    return _policy

Agent.epsilon_greedy_policy = epsilon_greedy_policy

In [12]:
def GLIE(max_time_step=1000, T=200):
    env = Env()
    agent = Agent()
    counts = np.zeros((WORLD_SIZE, WORLD_SIZE, N_ACTIONS), dtype=np.float32)
    Q = np.random.random((WORLD_SIZE, WORLD_SIZE, N_ACTIONS))
    epsilon = 0.99
    for k in range(1, max_time_step):
        epsilon *= 0.99
        policy = agent.epsilon_greedy_policy(Q, epsilon)
        init_s = agent.reset()
        episode = MCSample(env, policy, init_s, T=T)
        g = 0
        drop_step = 0
        for transition in episode[::-1]:
            s, a, r = transition['s'], transition['a'], transition['r']
            g = r + gamma * g
            drop_step += 1
            if drop_step > T / 2:
                counts[s[0], s[1], a] += 1
                Q[s[0], s[1], a] += 1.0 / counts[s[0], s[1], a] * (g - Q[s[0], s[1], a])
    return Q

In [13]:
optim_Q = GLIE(2000, 200)
optim_policy = np.argmax(optim_Q, axis=-1)
print(optim_policy)

[[3 0 2 2 2]
 [0 0 0 0 2]
 [3 0 0 0 0]
 [0 0 0 0 2]
 [3 0 0 0 2]]


![](http://ww2.sinaimg.cn/large/006tNc79ly1g5gi1r6k1oj30rp0ay3zw.jpg)

In [14]:
def greedy_Optim(max_time_step=1000, T=200):
    env = Env()
    agent = Agent()
    counts = np.zeros((WORLD_SIZE, WORLD_SIZE, N_ACTIONS), dtype=np.float32)
    Q = np.random.random((WORLD_SIZE, WORLD_SIZE, N_ACTIONS))
    for k in range(1, max_time_step):
        epsilon = -1
        policy = agent.epsilon_greedy_policy(Q, epsilon)
        init_s = agent.reset()
        episode = MCSample(env, policy, init_s, T=T)
        g = 0
        drop_step = 0
        for transition in episode[::-1]:
            s, a, r = transition['s'], transition['a'], transition['r']
            g = r + gamma * g
            drop_step += 1
            if drop_step > T / 2:
                counts[s[0], s[1], a] += 1
                Q[s[0], s[1], a] += 1.0 / counts[s[0], s[1], a] * (g - Q[s[0], s[1], a])
    return Q

In [15]:
optim_Q = greedy_Optim(3000, 200)
optim_policy = np.argmax(optim_Q, axis=-1)
print(optim_policy)

[[3 2 2 2 2]
 [3 0 1 0 0]
 [0 0 2 0 0]
 [1 0 2 0 0]
 [3 0 0 0 0]]


接下来我们实现增量式离策略每次拜访优化算法：

In [16]:
def increment_off_policy(max_time_step=1000, T=200):
    env = Env()
    agent = Agent()
    counts = np.zeros((WORLD_SIZE, WORLD_SIZE, N_ACTIONS), dtype=np.float32)
    Q = np.random.random((WORLD_SIZE, WORLD_SIZE, N_ACTIONS))
    for k in range(max_time_step):
        init_s = agent.reset()
        episode = MCSample(env, agent.random_policy, init_s, T=T)  # 使用随机策略mu进行探索
        g = 0
        W = 1
        drop_step = 0
        for transition in episode[::-1]:
            s, a, r = transition['s'], transition['a'], transition['r']
            g = r + gamma * g
            drop_step += 1
            if drop_step > T / 2:
                counts[s[0], s[1], a] += W
                Q[s[0], s[1], a] += W / counts[s[0], s[1], a] * (g - Q[s[0], s[1], a])
                if a != np.argmax(Q[s[0], s[1]]):
                    break
                W /= 0.25
    return Q

In [17]:
optim_Q = increment_off_policy(10000, 200)
optim_policy = np.argmax(optim_Q, axis=-1)
print(optim_policy)

[[3 3 2 1 2]
 [0 0 2 0 2]
 [0 0 3 0 0]
 [0 2 0 2 2]
 [3 0 0 0 0]]
